In [2]:
import torch
import numpy as np
from utils.generate_csbm import *
from utils.train import *
from utils.utils import *
from models.models import *
from models.setup import *
import networkx as nx
import pickle

In [3]:
train_ids = np.load("experiments/SNC/0.78phi/data/train_ids.npy")
val_ids = np.load("experiments/SNC/0.78phi/data/val_ids.npy")
test_ids = np.load("experiments//SNC/0.78phi/data/test_ids.npy")

for ith in range(20):
    csbm.generate_node_data(method="SNC", n_local=40)
    file_to_store = open("experiments/SNC/0.78phi/data/"+"csbm_"+str(ith)+".pickle", "wb")
    pickle.dump(csbm, file_to_store)
    file_to_store.close()

In [4]:
def exp_csbm(ith, method, folder_path, csbm, train_ids, val_ids, test_ids, 
             num_communication=100, batch_size=1,
             learning_rate=0.2, I=10, gradient=True, noise=False,
             Print=False, print_time=1,
             resume=False,
             bias=False,
             save=True):
    

    init_mlp = MLP(csbm.Xs[0].shape[1], 64, 2, bias)
        
    if (gradient == False):
        grad = "no_grad/"
    elif (gradient == True and noise==False):
        grad = "biased_grad/"
    elif (gradient == True and noise==True):
        grad = "noisy_grad/"
      
    A_tilde = calculate_Atilde(csbm.A, K=10, alpha=0.9)
    server = set_up_NC(csbm.Xs, csbm.ys, init_mlp, A_tilde, train_ids, val_ids, test_ids)
   
        
    tl, ta, vl, va = train_NC(server, num_communication, batch_size, learning_rate, I,
                              gradient, noise, 
                              Print, print_time)
    if save:    
        np.save(folder_path + method + grad + "I" + str(I) + "/tl_" + str(ith), tl)
        np.save(folder_path + method + grad + "I" + str(I) + "/ta_" + str(ith), ta)
        np.save(folder_path + method + grad + "I" + str(I) + "/vl_" + str(ith), vl)
        np.save(folder_path + method + grad + "I" + str(I) + "/va_" + str(ith), va)

        PATH = folder_path + method + grad + "I" + str(I) + "/model_" + str(ith)
        torch.save({
                'best_model_state_dict': server.best_cmodel.state_dict(),
                'learning_rate': learning_rate,
                'test_acc': server.eval_test()[1],
                'model_state_dict': server.cmodel.state_dict(),
                'best_valloss': server.best_valloss,
                'best_valacc': server.best_valacc,
                }, PATH)

In [18]:
torch.load("experiments/SNC/0.78phi/result/NC/biased_grad/I10/model_1",
            map_location=torch.device('cpu'))["test_acc"]

0.9210937499999998

In [6]:
for ith in range(20):
    for I in [10, 20, 50]:
        file_to_open= open("experiments/SNC/0.78phi/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(5000/I)
        exp_csbm(ith, "NC/", "experiments/SNC/0.78phi/result/", csbm, train_ids, val_ids, test_ids, 
             num_communication=nc, batch_size=40,
             learning_rate=0.2, I=I, gradient=True, noise=False,
             Print=False, print_time=10,
             resume=False,
             bias=False,
             save=True)
        print (ith, I)

0 10
0 20
0 50
1 10
1 20
1 50
2 10
2 20
2 50
3 10
3 20
3 50
4 10
4 20
4 50
5 10
5 20
5 50
6 10
6 20
6 50
7 10
7 20
7 50
8 10
8 20
8 50
9 10
9 20
9 50
10 10
10 20
10 50
11 10
11 20
11 50
12 10
12 20
12 50
13 10
13 20
13 50
14 10
14 20
14 50
15 10
15 20
15 50
16 10
16 20
16 50
17 10
17 20
17 50
18 10
18 20
18 50
19 10
19 20
19 50


In [15]:
for ith in range(20):
    for I in [10,20,50]:
        file_to_open= open("experiments/SNC/0.78phi/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(5000/I)
        exp_csbm(ith, "NC/", "experiments/SNC/0.78phi/result/", csbm, train_ids, val_ids, test_ids, 
             num_communication=nc, batch_size=40,
             learning_rate=0.2, I=I, gradient=False, noise=False,
             Print=False, print_time=10,
             resume=False,
             bias=False,
             save=True)
        print (ith, I)

0 10
0 20
0 50
1 10
1 20
1 50
2 10
2 20
2 50
3 10
3 20
3 50
4 10
4 20
4 50
5 10
5 20
5 50
6 10
6 20
6 50
7 10
7 20
7 50
8 10
8 20
8 50
9 10
9 20
9 50
10 10
10 20
10 50
11 10
11 20
11 50
12 10
12 20
12 50
13 10
13 20
13 50
14 10
14 20
14 50
15 10
15 20
15 50
16 10
16 20
16 50
17 10
17 20
17 50
18 10
18 20
18 50
19 10
19 20
19 50


In [22]:
for ith in range(20):
    for I in [1]:
        file_to_open= open("experiments/SNC/0.78phi/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(5000/I)
        exp_csbm(ith, "NC/", "experiments/SNC/0.78phi/result/", csbm, train_ids, val_ids, test_ids, 
             num_communication=nc, batch_size=40,
             learning_rate=0.2, I=I, gradient=True, noise=False,
             Print=False, print_time=10,
             resume=False,
             bias=False,
             save=True)
        print (ith, I)

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1


In [19]:
for ith in range(20):
    for I in [10,20,50]:
        file_to_open= open("experiments/SNC/0.78phi/data/"+"csbm_"+str(ith)+".pickle", "rb")
        csbm = pickle.load(file_to_open)
        file_to_open.close()
        nc = int(5000/I)
        exp_csbm(ith, "NC/", "experiments/SNC/0.78phi/result/", csbm, train_ids, val_ids, test_ids, 
             num_communication=nc, batch_size=40,
             learning_rate=0.2, I=I, gradient=True, noise=True,
             Print=False, print_time=10,
             resume=False,
             bias=False,
             save=True)
        print (ith, I)

0 10
0 20
0 50
1 10
1 20
1 50
2 10
2 20
2 50
3 10
3 20
3 50
4 10
4 20
4 50
5 10
5 20
5 50
6 10
6 20
6 50
7 10
7 20
7 50
8 10
8 20
8 50
9 10
9 20
9 50
10 10
10 20
10 50
11 10
11 20
11 50
12 10
12 20
12 50
13 10
13 20
13 50
14 10
14 20
14 50
15 10
15 20
15 50
16 10
16 20
16 50
17 10
17 20
17 50
18 10
18 20
18 50
19 10
19 20
19 50
